# Comprehensive Analysis and Transformation of the English Women's Soccer League Data Using Spark

## Overview

This notebook processes match data for the Women's Super League (WSL) from a bronze data lake, and stores the processed data in a silver data lake. 

The process performs various transformations and calculations to derive useful statistics and rankings for the teams, providing insights into team performances across seasons, focusing on matches between 2019 and 2023.

## Step 1

### 1. Retrieve Configuration Details

Retrieve mount points for bronze and silver data lakes from the 'config' notebook.

In [0]:
# Retrieve configuration details, including mount points for bronze and silver data lakes

config = dbutils.notebook.run('../storage/01-config', 180)
config_dict = eval(config)

bronzeMountPoint = config_dict["bronzeMountPoint"]
silverMountPoint = config_dict["silverMountPoint"]

### 2. Read and Display Data

Load and display the data previously stored in the bronze data lake.

In [0]:
from pyspark.sql.functions import col, sum, desc, rank
from pyspark.sql import Window

In [0]:
# Reading and displaying data from the bronze mount point

dfMatches = spark.read.format('delta').load(bronzeMountPoint + '/women_football_matches')
display(dfMatches.limit(50))

dfMatches.printSchema()

MatchId,League,Season,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,MatchResult,MatchDate,HomeWin,AwayWin,Draw
101658,D1 FFF,2019,Bordeaux,Fleury,4,1,H,24-08-19,1,0,0
102633,D1 FFF,2019,Guingamp,Metz,2,0,H,24-08-19,1,0,0
105384,D1 FFF,2019,Paris FC,Dijon,3,1,H,24-08-19,1,0,0
107191,D1 FFF,2019,Montpellier,Stade de Reims,2,0,H,24-08-19,1,0,0
107631,D1 FFF,2019,Lyon,Ol. Marseille,6,0,H,24-08-19,1,0,0
108229,D1 FFF,2019,Paris S-G,Soyaux,7,0,H,25-08-19,1,0,0
109491,D1 FFF,2019,Fleury,Montpellier,1,3,A,07-09-19,0,1,0
109961,D1 FFF,2019,Ol. Marseille,Guingamp,2,1,H,07-09-19,1,0,0
113735,D1 FFF,2019,Soyaux,Paris FC,0,0,D,07-09-19,0,0,1
116885,D1 FFF,2019,Stade de Reims,Lyon,3,8,A,07-09-19,0,1,0


root
 |-- MatchId: integer (nullable = true)
 |-- League: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- HomeTeam: string (nullable = true)
 |-- AwayTeam: string (nullable = true)
 |-- HomeTeamGoals: integer (nullable = true)
 |-- AwayTeamGoals: integer (nullable = true)
 |-- MatchResult: string (nullable = true)
 |-- MatchDate: string (nullable = true)
 |-- HomeWin: integer (nullable = true)
 |-- AwayWin: integer (nullable = true)
 |-- Draw: integer (nullable = true)



### 3. Filter and Select Relevant Columns

Filter the data to focus on WSL matches between 2019 and 2023, selecting specific columns for analysis.

In [0]:
# Filtering and selecting specific columns from a DataFrame to focus on Women's Soccer League matches between 2019 and 2023.

dfWomenSoccerLeague = dfMatches.filter((col('League') == 'WSL') & \
    (col('Season') >= 2019) & \
    (col('Season') <= 2023)
)

dfWomenSoccerLeague = dfWomenSoccerLeague.select('MatchId', 'Season', 'MatchDate', 'HomeTeam', 'AwayTeam', 'HomeTeamGoals', 'AwayTeamGoals', 'MatchResult', 'HomeWin', 'AwayWin', 'Draw')

display(dfWomenSoccerLeague.limit(20))

dfWomenSoccerLeague.count()

MatchId,Season,MatchDate,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,MatchResult,HomeWin,AwayWin,Draw
436880,2019,07-09-19,Manchester City,Manchester Utd,1,0,H,1,0,0
439058,2019,07-09-19,Bristol City,Brighton,0,0,D,0,0,1
441056,2019,08-09-19,Chelsea,Tottenham,1,0,H,1,0,0
441268,2019,08-09-19,Birmingham City,Everton,0,1,A,0,1,0
441488,2019,08-09-19,Liverpool,Reading,0,1,A,0,1,0
441780,2019,08-09-19,Arsenal,West Ham,2,1,H,1,0,0
442346,2019,15-09-19,Reading,Manchester City,0,2,A,0,1,0
442873,2019,15-09-19,Everton,Bristol City,2,0,H,1,0,0
446456,2019,15-09-19,Tottenham,Liverpool,1,0,H,1,0,0
447682,2019,15-09-19,Brighton,Chelsea,1,1,D,0,0,1


615

### 4. Aggregate Home and Away Matches

Aggregate data separately for home and away matches, calculating wins, draws, losses, goals scored, and conceded.

In [0]:
# Grouping the filtered WSL matches by season and home team, aggregating various statistics such as wins, draws, losses, goals scored, and conceded.

dfWSLHomeMatches = dfWomenSoccerLeague.groupBy('Season', 'HomeTeam') \
    .agg(
        sum('HomeWin').alias('TotalHomeWins'),
        sum('Draw').alias('TotalHomeDraws'),
        sum('AwayWin').alias('TotalHomeLosses'),
        sum('HomeTeamGoals').alias('TotalHomeGoalsScored'),
        sum('AwayTeamGoals').alias('TotalHomeGoalsConceded')
    ) \
    .withColumnRenamed('HomeTeam', 'Team')

display(dfWSLHomeMatches.limit(20))

Season,Team,TotalHomeWins,TotalHomeDraws,TotalHomeLosses,TotalHomeGoalsScored,TotalHomeGoalsConceded
2020,West Ham,0,4,7,6,22
2023,Liverpool,6,2,3,19,14
2020,Aston Villa,1,3,7,5,25
2022,Arsenal,8,1,2,24,8
2019,Everton,4,0,3,13,9
2021,West Ham,3,4,4,14,15
2022,Aston Villa,4,2,5,22,22
2022,Leicester City,3,1,7,11,22
2020,Reading,2,5,4,11,20
2023,Aston Villa,1,3,7,13,28


In [0]:
# Grouping the filtered WSL matches by season and away team, aggregating various statistics such as wins, draws, losses, goals scored, and conceded.

dfWSLAwayMatches = dfWomenSoccerLeague.groupBy('Season', 'AwayTeam') \
    .agg(
        sum('AwayWin').alias('TotalAwayWins'),
        sum('Draw').alias('TotalAwayDraws'),
        sum('HomeWin').alias('TotalAwayLosses'),
        sum('AwayTeamGoals').alias('TotalAwayGoalsScored'),
        sum('HomeTeamGoals').alias('TotalAwayGoalsConceded'),
    ) \
    .withColumnRenamed('AwayTeam', 'Team')

display(dfWSLAwayMatches.limit(20))

Season,Team,TotalAwayWins,TotalAwayDraws,TotalAwayLosses,TotalAwayGoalsScored,TotalAwayGoalsConceded
2020,West Ham,3,2,6,15,17
2023,Liverpool,6,3,2,17,14
2020,Aston Villa,2,3,6,10,22
2022,Arsenal,7,1,3,25,8
2019,Everton,2,1,4,8,12
2021,West Ham,4,2,5,9,18
2022,Aston Villa,7,2,2,25,15
2022,Leicester City,2,0,9,4,26
2020,Reading,3,4,4,14,21
2023,Aston Villa,6,0,5,14,15


### 5. Combine Aggregated Data

Join home and away match data to create a comprehensive view of each team's performance.

In [0]:
# Joining the aggregated home and away match dataframes on season and team names to create a comprehensive view of each team's performance across seasons.

dfWSLTable = dfWSLHomeMatches.join(dfWSLAwayMatches, ['Season', 'Team'], 'inner')

display(dfWSLTable)

Season,Team,TotalHomeWins,TotalHomeDraws,TotalHomeLosses,TotalHomeGoalsScored,TotalHomeGoalsConceded,TotalAwayWins,TotalAwayDraws,TotalAwayLosses,TotalAwayGoalsScored,TotalAwayGoalsConceded
2020,West Ham,0,4,7,6,22,3,2,6,15,17
2023,Liverpool,6,2,3,19,14,6,3,2,17,14
2020,Aston Villa,1,3,7,5,25,2,3,6,10,22
2022,Arsenal,8,1,2,24,8,7,1,3,25,8
2019,Everton,4,0,3,13,9,2,1,4,8,12
2021,West Ham,3,4,4,14,15,4,2,5,9,18
2022,Aston Villa,4,2,5,22,22,7,2,2,25,15
2022,Leicester City,3,1,7,11,22,2,0,9,4,26
2020,Reading,2,5,4,11,20,3,4,4,14,21
2023,Aston Villa,1,3,7,13,28,6,0,5,14,15


### 6. Calculate Performance Metrics

Add calculated columns for total games played, won, drawn, lost, goals scored, and conceded.

In [0]:
# Adding calculated columns for total games played, won, drawn, lost, goals scored ('For'), and goals conceded ('Against').

dfWSLTable = dfWSLTable.withColumn('Played', col('TotalHomeWins') + col('TotalAwayWins') \
    + col('TotalHomeDraws') + col('TotalAwayDraws') + col('TotalHomeLosses') + col('TotalAwayLosses')) \
    .withColumn('Won', col('TotalHomeWins') + col('TotalAwayWins')) \
    .withColumn('Drawn', col('TotalHomeDraws') + col('TotalAwayDraws')) \
    .withColumn('Lost', col('TotalHomeLosses') + col('TotalAwayLosses')) \
    .withColumn('For', col('TotalHomeGoalsScored') + col('TotalAwayGoalsScored')) \
    .withColumn('Against', col('TotalHomeGoalsConceded') + col('TotalAwayGoalsConceded')) \
 
display(dfWSLTable.limit(20))

Season,Team,TotalHomeWins,TotalHomeDraws,TotalHomeLosses,TotalHomeGoalsScored,TotalHomeGoalsConceded,TotalAwayWins,TotalAwayDraws,TotalAwayLosses,TotalAwayGoalsScored,TotalAwayGoalsConceded,Played,Won,Drawn,Lost,For,Against
2020,West Ham,0,4,7,6,22,3,2,6,15,17,22,3,6,13,21,39
2023,Liverpool,6,2,3,19,14,6,3,2,17,14,22,12,5,5,36,28
2020,Aston Villa,1,3,7,5,25,2,3,6,10,22,22,3,6,13,15,47
2022,Arsenal,8,1,2,24,8,7,1,3,25,8,22,15,2,5,49,16
2019,Everton,4,0,3,13,9,2,1,4,8,12,14,6,1,7,21,21
2021,West Ham,3,4,4,14,15,4,2,5,9,18,22,7,6,9,23,33
2022,Aston Villa,4,2,5,22,22,7,2,2,25,15,22,11,4,7,47,37
2022,Leicester City,3,1,7,11,22,2,0,9,4,26,22,5,1,16,15,48
2020,Reading,2,5,4,11,20,3,4,4,14,21,22,5,9,8,25,41
2023,Aston Villa,1,3,7,13,28,6,0,5,14,15,22,7,3,12,27,43


### 7. Clean Up and Sort

Remove intermediate aggregation columns and sort the table by season.

In [0]:
# Removing unnecessary intermediate aggregation columns and sorting the final table by season in descending order.

removeCols = ['TotalHomeWins', 'TotalHomeDraws', 'TotalHomeLosses', 'TotalHomeGoalsScored', 'TotalHomeGoalsConceded',
              'TotalAwayWins', 'TotalAwayDraws', 'TotalAwayLosses', 'TotalAwayGoalsScored', 'TotalAwayGoalsConceded'
             ]

dfWSLTable = dfWSLTable.drop(*removeCols).sort(col('Season').desc())

display(dfWSLTable.limit(20))

Season,Team,Played,Won,Drawn,Lost,For,Against
2023,Manchester City,22,18,1,3,61,15
2023,Liverpool,22,12,5,5,36,28
2023,Aston Villa,22,7,3,12,27,43
2023,Arsenal,22,16,2,4,53,20
2023,Manchester Utd,22,10,5,7,42,32
2023,Chelsea,22,18,1,3,71,18
2023,Brighton,22,5,4,13,26,48
2023,Leicester City,22,4,6,12,26,45
2023,West Ham,22,3,6,13,20,45
2023,Bristol City,22,1,3,18,20,70


### 8. Goal Difference and Points

Calculate goal difference and points based on wins and draws.

In [0]:
# Adding a new column 'GD' to represent the goal difference, calculated as the difference between goals scored ('For') and goals conceded ('Against').

dfWSLTable = dfWSLTable.withColumn('GD', col('For') - col('Against'))

display(dfWSLTable.limit(20))

Season,Team,Played,Won,Drawn,Lost,For,Against,GD
2023,Arsenal,22,16,2,4,53,20,33
2023,Liverpool,22,12,5,5,36,28,8
2023,Aston Villa,22,7,3,12,27,43,-16
2023,Manchester City,22,18,1,3,61,15,46
2023,Everton,22,6,5,11,24,37,-13
2023,Chelsea,22,18,1,3,71,18,53
2023,Brighton,22,5,4,13,26,48,-22
2023,Leicester City,22,4,6,12,26,45,-19
2023,West Ham,22,3,6,13,20,45,-25
2023,Bristol City,22,1,3,18,20,70,-50


In [0]:
# Calculating a 'Points' column based on the number of wins and draws, following the standard scoring system where three points are awarded for a win and one point for a draw.

dfWSLTable = dfWSLTable.withColumn('Points', (col('Won') * 3) + (col('Drawn') * 1))

display(dfWSLTable.limit(20))

Season,Team,Played,Won,Drawn,Lost,For,Against,GD,Points
2023,Manchester City,22,18,1,3,61,15,46,55
2023,Liverpool,22,12,5,5,36,28,8,41
2023,Aston Villa,22,7,3,12,27,43,-16,24
2023,Arsenal,22,16,2,4,53,20,33,50
2023,Manchester Utd,22,10,5,7,42,32,10,35
2023,Chelsea,22,18,1,3,71,18,53,55
2023,Brighton,22,5,4,13,26,48,-22,19
2023,Leicester City,22,4,6,12,26,45,-19,18
2023,West Ham,22,3,6,13,20,45,-25,15
2023,Bristol City,22,1,3,18,20,70,-50,6


### 9. Rank Teams

Assign positions to teams within each season based on points, goal difference, and goals scored.

In [0]:
# Assigning positions to teams within each season based on points, goal difference, and goals scored, using a window specification. 

windowSpec = Window.partitionBy('Season').orderBy(
    desc('Points'),
    desc('GD'),
    desc('For')
)

dfWSLTable = dfWSLTable.withColumn('Position', rank().over(windowSpec))

dfWSLTable = dfWSLTable.select('Position', 'Season', 'Team', 'Played', 'Won', 'Drawn', 'Lost', 'For', 'Against', 'GD', 'Points')

display(dfWSLTable.limit(20))

Position,Season,Team,Played,Won,Drawn,Lost,For,Against,GD,Points
1,2019,Manchester City,16,13,1,2,39,9,30,40
2,2019,Chelsea,15,12,3,0,47,11,36,39
3,2019,Arsenal,15,12,0,3,40,13,27,36
4,2019,Manchester Utd,14,7,2,5,24,12,12,23
5,2019,Reading,14,6,3,5,21,24,-3,21
6,2019,Tottenham,15,6,2,7,15,24,-9,20
7,2019,Everton,14,6,1,7,21,21,0,19
8,2019,West Ham,14,5,1,8,19,34,-15,16
9,2019,Brighton,16,3,4,9,11,30,-19,13
10,2019,Bristol City,14,2,3,9,9,38,-29,9


### 10. Save Processed Data

Save the final processed league table DataFrame to Delta Lake, overwriting existing data.

In [0]:
# Saving the final processed league table DataFrame to a Delta Lake format file, overwriting any existing data at the specified path.

dfWSLTable.write.format('delta').mode('overwrite').save(silverMountPoint + '/WSL/' + 'league_table')